# Inference

In [ ]:
# # Torch - Torchvision - IceVision - IceData - MMDetection - YOLOv5 - EfficientDet Installation
# !wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh

# # Choose your installation target: cuda11 or cuda10 or cpu
# !bash icevision_install.sh cuda11 master

## Imports
All of the IceVision components can be easily imported with a single line.


In [ ]:
from icevision.all import *

## List of images for inference

Please store your images in a folder, and populate the `path_to_folder` variable with the corresponding folder name.

In [ ]:
# Pick your images folder
path_to_image_folder = "../data/images"
img_files = get_image_files(path_to_image_folder)[52:]
# img_files

In [ ]:
img = PIL.Image.open(img_files[0])
img

## Loading a checkpoint and creating the corresponding model

The checkpoint file can be either a local file or an URL

In [ ]:
checkpoint_path = '../models/2022-07-27_resnet_slates_50.pth'

In [ ]:
from icevision.models import *

checkpoint_and_model = model_from_checkpoint(checkpoint_path, map_location='cuda')

In [ ]:
# Just logging the info
model_type = checkpoint_and_model["model_type"]
backbone = checkpoint_and_model["backbone"]
class_map = checkpoint_and_model["class_map"]
img_size = checkpoint_and_model["img_size"]

In [ ]:
model_type

In [ ]:
backbone

In [ ]:
class_map

In [ ]:
img_size

## Get Model Object

`model_from_checkpoint(checkpoint_path)` returns a dictionary: `checkpoint_and_model`. The model object is stored in `checkpoint_and_model["model"]`. 

In [ ]:
# Get model object
# The model is automatically set in the evaluation mode
model = checkpoint_and_model["model"]

# Check device
device=next(model.parameters()).device
device

In [ ]:
model.to('cuda')

device=next(model.parameters()).device
device

In [ ]:
# Transforms
img_size = checkpoint_and_model["img_size"]
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(img_size), tfms.A.Normalize()])

## Single Image Inference

The `end2end_detect()` not only compute predictions for a single image but also automatically adjust predicted boxes to the original image size

In [ ]:
img = PIL.Image.open(img_files)
pred_dict  = model_type.end2end_detect(img, valid_tfms, model, class_map=class_map, detection_threshold=0.7)

pred_dict['img']

## Batch Inference

The following option shows to do generate inference for a set of images. The latter is processed in batches.

In [ ]:
# Create a dataset
imgs_array = [PIL.Image.open(Path(fname)) for fname in img_files]
infer_ds = Dataset.from_images(imgs_array, valid_tfms, class_map=class_map)

# Batch Inference
infer_dl = model_type.infer_dl(infer_ds, batch_size=4, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True, detection_threshold=0.50)

In [ ]:
# You may need to un-comment this in certain environments to preview images:
%matplotlib inline
show_preds(preds=preds[0:5])

## How to export inferences as COCO annotations

These will match the dimensions of the original images. This could be useful if you'd like to create preliminary pseudo-annotations for unlabeled data:

In [ ]:
from icevision.data.convert_records_to_coco_style import export_batch_inferences_as_coco_annotations

In [ ]:
info = {
    "year": "2022",
    "version": "1",
    "description": "Exported from IceVision",
    "contributor": "Awesome contributor",
    "url": "https://lazyannotator.fun",
    "date_created": "2022-08-05T20:13:09+00:00"
}

In [ ]:
licenses = [
    {
        "name": "Creative Commons Attribution 4.0",
        "id": 0,
        "url": "https://creativecommons.org/licenses/by/4.0/legalcode",
    }
]

In [ ]:
export_batch_inferences_as_coco_annotations(
    preds=preds,
    img_files=img_files,
    transforms=valid_tfms,
    class_map=class_map,
    output_filepath="../data/annotations/2022-09-27_preds_to_check_then_combine.json",
    info=info,
    licenses=licenses,
)

### Preview predictions on original image:

In [ ]:
# Index of image you'd like to check
i = 33

this_pred = preds[i]
record = this_pred.pred

# Draw that image
pred_img = draw_record(
        record=this_pred,
        class_map=class_map,
        display_label=True,
        display_score=True,
        display_bbox=True,
        font_path=get_default_font(),
        font_size=12,
        label_color=("#FF59D6"),
        return_as_pil_img=True,
    )

pred_img